In [2]:
!pip list

Package                   Version
------------------------- ------------
accelerate                1.4.0
agno                      1.1.8
aiohappyeyeballs          2.4.6
aiohttp                   3.11.12
aiosignal                 1.3.2
aiosqlite                 0.21.0
alembic                   1.14.1
annotated-types           0.7.0
anyio                     4.6.0
apprise                   1.9.2
asgi-lifespan             2.1.0
asttokens                 3.0.0
async-timeout             4.0.3
asyncpg                   0.29.0
attrs                     25.1.0
bitsandbytes              0.45.3
bleach                    6.2.0
bokeh                     3.6.3
cachetools                5.5.2
certifi                   2025.1.31
cffi                      1.17.1
charset-normalizer        3.4.1
click                     8.1.8
cloudpickle               3.1.1
colorama                  0.4.6
comm                      0.2.2
contourpy                 1.3.1
coolname                  2.2.0
croniter           

In [3]:
!pip install pdfplumber


[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip



   ---------------------------------------- 0.0/5.6 MB ? eta -:--:--
   ---------------------------------------- 5.6/5.6 MB 85.0 MB/s eta 0:00:00
   ---------------------------------------- 0.0/2.9 MB ? eta -:--:--
   ---------------------------------------- 2.9/2.9 MB 82.2 MB/s eta 0:00:00


In [4]:
pwd

'c:\\Users\\sufya\\Documents\\expense_tracker\\notebook'

In [14]:
pdf_path = "../data/raw/dbs_unbilled.pdf"

import os

os.path.basename(pdf_path).split('.')[0]

'dbs_unbilled'

In [20]:
# Let's parse the uploaded DBS PDF, extract the desired transactions, 
# and show a clean table with (date, description, amount). 
# We'll also save it as a CSV for you.
import re
import pdfplumber
import pandas as pd
from datetime import datetime

pdf_path = "../data/raw/dbs_unbilled.pdf"


def parse_amount(text):
    """
    Parse an amount string like 'S$248.80' or 'S$857.62 cr'.
    Returns a signed float (credits become negative).
    """
    # Remove any commas and NBSPs
    t = text.replace(",", "").replace("\xa0", " ").strip()
    # Detect 'cr' for credit (negative)
    is_credit = t.lower().endswith(" cr")
    # Grab the numeric part after 'S$'
    m = re.search(r"S\$\s*([0-9]+(?:\.[0-9]{1,2})?)", t)
    if not m:
        return None
    amt = float(m.group(1))
    if is_credit:
        amt = -amt
    return amt

def is_date_line(line):
    # Matches '12 Aug 2025 ...' at start
    return re.match(r"^\s*\d{1,2}\s+[A-Za-z]{3}\s+\d{4}\b", line) is not None

def extract_date_desc_amt(line):
    # Split into 3 parts: date, description, amount
    # Amount is the last 'S$...' occurrence
    m_amount = list(re.finditer(r"S\$\s*[0-9]+(?:\.[0-9]{1,2})?(?:\s*cr)?\s*$", line, flags=re.IGNORECASE))
    if not m_amount:
        return None
    m = m_amount[-1]
    amount_str = line[m.start():m.end()]
    left = line[:m.start()].rstrip()

    # Date at start
    m_date = re.match(r"^\s*(\d{1,2}\s+[A-Za-z]{3}\s+\d{4})\b", left)
    if not m_date:
        return None
    date_str = m_date.group(1)
    desc = left[m_date.end():].strip()
    # Clean odd characters (like replacement chars from PDF)
    desc = desc.replace("\xa0", " ").replace("�", "")

    # Parse date to ISO
    try:
        date_iso = datetime.strptime(date_str, "%d %b %Y").date().isoformat()
    except Exception:
        date_iso = date_str  # fallback to original

    amount = parse_amount(amount_str)
    return date_iso, desc, amount

rows = []

with pdfplumber.open(pdf_path) as pdf:
    for page in pdf.pages:
        text = page.extract_text() or ""
        print(text.split('\n'))
        # Normalize whitespace for consistent parsing
        lines = [ln.strip() for ln in text.splitlines() if ln.strip()]

        # Skip entire sections we don't want:
        # - lines containing 'DBS Points' table (we'll just never match them as transactions)
        # - 'PAYMENT - DBS INTERNET/WIRELESS' and its 'Sub-Total' row
        skip_payment_section = False
        for i, line in enumerate(lines):
            # Detect start of the "Payment - DBS Internet/Wireless" block
            if "PAYMENT - DBS INTERNET/WIRELESS" in line.upper():
                skip_payment_section = True
                continue
            if skip_payment_section:
                # End skipping when we hit a 'Sub-Total' line or a blank separator or a new header
                if line.upper().startswith("SUB-TOTAL") or line.upper().startswith("TRANSACTION DATE") or "DBS ALTITUDE" in line.upper():
                    skip_payment_section = False
                continue  # while skipping, ignore

            # Only consider lines that look like actual transaction rows
            if is_date_line(line) and ("S$" in line):
                out = extract_date_desc_amt(line)
                if out is not None:
                    date_iso, desc, amount = out
                    # Basic guard: ignore if description looks like other headers or tables
                    if desc.upper().startswith(("DBS ALTITUDE", "TRANSACTION DATE", "SUB-TOTAL")):
                        continue
                    rows.append({"transaction_date": date_iso, "description": desc, "amount_sgd": amount})

# Build DataFrame
df = pd.DataFrame(rows, columns=["transaction_date", "description", "amount_sgd"]).sort_values(by="transaction_date")

# name of new file 
new_name = os.path.basename(pdf_path).split('.')[0]

# output dir 
output_dir = "../data/processed"

# Save to CSV
csv_path = os.path.join(output_dir, f"{new_name}_parsed.csv")
df.to_csv(csv_path, index=False)

['This is a print preview page', 'Close this window.', 'View Transaction History', '30 Aug 2025 07:10 AM Singapore', 'Modify Search', 'DBS Altitude Visa Signature Card4119-1100-9323-8894', 'Credit Limit Available Limit DBS Points', 'S$19,100.00 S$16,351.00 19950', 'Unbilled Transactionsas per statement printed on 30 Aug 2025', 'Transaction Date Description Amount', '24 Aug 2025 PAYMENT - DBS INTERNET/WIRELESS S$857.62 cr', 'Sub-Total -S$ 857.62', 'Transaction Date Description Amount', 'DBS Altitude Visa Signature Card4119-1100-9323-8894', '12 Aug 2025 MALAY AIR2322479723900 PTB 2 SINGAPO SG S$14.00']
['12 Aug 2025 MALAY AIR2322479723901 PTB 2 SINGAPO SG S$248.80', '12 Aug 2025 MALAY AIR2322479723902 PTB 2 SINGAPO SG S$248.80', '15 Aug 2025 CARDUP-PRUDENTIAL ASSU SINGAPORE SG S$125.72', '25 Aug 2025 PRUDENTIAL 733 SINGAPORE SG S$17.77', '25 Aug 2025 PUNGGOL TOWN COUNCIL SINGAPORE SG S$69.40', '28 Aug 2025 CARDUP-PRUDENTIAL ASSU SINGAPORE SG S$305.55', '28 Aug 2025 CARDUP-PRUDENTIAL ASSU

In [22]:
test = ['This is a print preview page', 'Close this window.', 'View Transaction History', '30 Aug 2025 07:10 AM Singapore', 'Modify Search', 'DBS Altitude Visa Signature Card4119-1100-9323-8894', 'Credit Limit Available Limit DBS Points', 'S$19,100.00 S$16,351.00 19950', 'Unbilled Transactionsas per statement printed on 30 Aug 2025', 'Transaction Date Description Amount', '24 Aug 2025 PAYMENT - DBS INTERNET/WIRELESS S$857.62 cr', 'Sub-Total -S$ 857.62', 'Transaction Date Description Amount', 'DBS Altitude Visa Signature Card4119-1100-9323-8894', '12 Aug 2025 MALAY AIR2322479723900 PTB 2 SINGAPO SG S$14.00']

In [ ]:
import re

for t in test:
    if 'DBS Altitude' in t:
        words = re.findall(r'\b\w+\b', t)
        filtered_words = [word for word in words if word in ['DBS', 'Altitude', 'Visa', 'Signature', 'Card']]
        print(' '.join(filtered_words))               

DBS Altitude Visa Signature
DBS Altitude Visa Signature


In [35]:

import pdfplumber
with pdfplumber.open('../data/raw/dbs_unbilled.pdf') as pdf:
    text_content = []
    for page in pdf.pages:
        text = page.extract_text() or ""
        # Normalize whitespace for consistent parsing
        lines = [ln.strip() for ln in text.splitlines() if ln.strip()]
        text_content.extend(lines)
rows = []


# Skip entire sections we don't want:
# - lines containing 'DBS Points' table (we'll just never match them as transactions)
# - 'PAYMENT - DBS INTERNET/WIRELESS' and its 'Sub-Total' row
skip_payment_section = False
for i, line in enumerate(text_content):
    # Detect start of the "Payment - DBS Internet/Wireless" block
    if "PAYMENT - DBS INTERNET/WIRELESS" in line.upper():
        skip_payment_section = True
        continue
    if skip_payment_section:
        # End skipping when we hit a 'Sub-Total' line or a blank separator or a new header
        if line.upper().startswith("SUB-TOTAL") or line.upper().startswith("TRANSACTION DATE") or "DBS ALTITUDE" in line.upper():
            skip_payment_section = False
        continue  # while skipping, ignore

    # Only consider lines that look like actual transaction rows
    if is_date_line(line) and ("S$" in line):
        out = extract_date_desc_amt(line)
        if out is not None:
            date_iso, desc, amount = out
            # Basic guard: ignore if description looks like other headers or tables
            if desc.upper().startswith(("DBS ALTITUDE", "TRANSACTION DATE", "SUB-TOTAL")):
                continue
            rows.append({"transaction_date": date_iso, "description": desc, "amount_sgd": amount})

# Build DataFrame
df = pd.DataFrame(rows, columns=["transaction_date", "description", "amount_sgd"]).sort_values(by="transaction_date")

In [37]:
text_content

['This is a print preview page',
 'Close this window.',
 'View Transaction History',
 '30 Aug 2025 07:10 AM Singapore',
 'Modify Search',
 'DBS Altitude Visa Signature Card4119-1100-9323-8894',
 'Credit Limit Available Limit DBS Points',
 'S$19,100.00 S$16,351.00 19950',
 'Unbilled Transactionsas per statement printed on 30 Aug 2025',
 'Transaction Date Description Amount',
 '24 Aug 2025 PAYMENT - DBS INTERNET/WIRELESS S$857.62 cr',
 'Sub-Total -S$ 857.62',
 'Transaction Date Description Amount',
 'DBS Altitude Visa Signature Card4119-1100-9323-8894',
 '12 Aug 2025 MALAY AIR2322479723900 PTB 2 SINGAPO SG S$14.00',
 '12 Aug 2025 MALAY AIR2322479723901 PTB 2 SINGAPO SG S$248.80',
 '12 Aug 2025 MALAY AIR2322479723902 PTB 2 SINGAPO SG S$248.80',
 '15 Aug 2025 CARDUP-PRUDENTIAL ASSU SINGAPORE SG S$125.72',
 '25 Aug 2025 PRUDENTIAL 733 SINGAPORE SG S$17.77',
 '25 Aug 2025 PUNGGOL TOWN COUNCIL SINGAPORE SG S$69.40',
 '28 Aug 2025 CARDUP-PRUDENTIAL ASSU SINGAPORE SG S$305.55',
 '28 Aug 2025 CA

In [43]:
def check_dbs(raw_content: list) -> bool:
    import re

    for t in raw_content:
        if 'DBS Altitude' in t:
            return True
    return False
            

check_dbs(text_content)

True